In [2]:
import core.backup as bck
import core.constants as cst
import core.correction as cor
import core.reduction as red
import core.pending as pd
import core.utils as ut
from core.plugin import Plugin

# DEBUG-ONLY
from importlib import reload
mods = [bck, cst, cor, red, pd, ut]
for mod in mods:
    reload(mod)
        
#---------------------------------------------------#

import argparse
from datetime import datetime
import inspect
import os
import pkgutil
import sys
import time
import warnings

class PluginCollector(object):
    """ Upon creation, this class will read the plugins
        package for modules that contain a class definition
        that is inheriting from the Plugin class.
    """

    def __init__(self):
        """ Constructor that initiates the reading of all available
            plugins when an instance of the PluginCollector object
            is created.
        """
        self.plugin_package = "plugins"
        self.plugins = []
        self.find_plugins()

    def find_plugins(self):
        """ Find and store all Plugin-inherited plugins stored in
            the '/plugins' folder. Append them to the list in the 
            attribute plugins.
        """
        imported_package = __import__(self.plugin_package, fromlist=[""])

        for _, pluginname, ispkg in pkgutil.iter_modules(imported_package.__path__, imported_package.__name__ + '.'):
            if not ispkg:
                plugin_module = __import__(pluginname, fromlist=[""])
                clsmembers = inspect.getmembers(plugin_module, inspect.isclass)
                for (_, c) in clsmembers:
                    # Only add classes that are a sub class of Plugin, but NOT Plugin itself
                    if issubclass(c, Plugin) & (c is not Plugin):
                        self.plugins.append(c())

def load_args():
    """ Function that specifies the expected arguments
        given on the command line by the user.
    """
    
    # Sets the description that will be shown with --help
    parser = argparse.ArgumentParser(description="Program that updates the Blaauw Observatory \
                                                  database by automatically reducing the fits files")
    
    # Add required argument for path to observation folder
    target = parser.add_mutually_exclusive_group()
    target.add_argument("-f", "--folder", type=str, help="The absolute path to the observation directory")
    target.add_argument("-d", "--date", type=str, help="The observation date in yy-mm-dd")
    target.add_argument("-u", "--update", action='store_const', const=datetime.today().strftime("%y-%m-%d"),
                        dest="date", help="Select todays directory if it exists")
    
    # Add arguments for the intended action
    parser.add_argument("-b", "--backup", action="store_true", help="Move a copy of the raw data to local folder")
    parser.add_argument("-c", "--correction", action="store_true", help="Save raw correction frames locally")
    parser.add_argument("-r", "--reduce", action="store_true", help="Reduce all the present light frames")
    #parser.add_argument("-p", "--preview", action="store_true", help="Create a .png preview per light file")
    parser.add_argument("-p", "--pending", action="store_true", help="Rerun pending reductions")
    parser.add_argument("-m", "--modules", action="store_true", help="Run external modules stored in /plugins")
    
    # Add argument for verbosity
    parser.add_argument("-v", "--verbose", action="store_true", help="Increase the amount of output")
    
    # Retrieve the passed arguments and check their validity
    args = parser.parse_args()
    targets = ut.check_args(parser, args)
    
    return args, targets

def run_plugins(obs, working_dir, args):
    # Collect all plugins
    loaded_plugins = PluginCollector()
    # Execute each plugin
    for plugin in loaded_plugins.plugins:
        ut.Print(f"Running external plugin: {plugin.title}", args)
        plugin.on_run(obs, working_dir, args)
        ut.Print(f"{plugin.title} executed", args)
    
def run_plugins_single(obs, working_dir, args, file, called_from):
    # Collect all plugins
    loaded_plugins = PluginCollector()
    # Execute each plugin for a specific file only
    for plugin in loaded_plugins.plugins:
        # Only rerun plugin if specified by plugin
        # TODO: not really elegant to compare strings...
        if (called_from == "pending" and plugin.rerun_on_pending) or \
           (called_from == "astromety" and plugin.rerun_on_astrometry):
            ut.Print(f"(Re)running external plugin: {plugin.title}", args)
            plugin.on_run_single(obs, working_dir, args, file)
            ut.Print(f"{plugin.title} (re-)executed", args)

def run_backup(obs, working_dir, args):
    ut.Print("Creating backup...", args, True)
    bck.create_backup(obs, working_dir, args)
    ut.Print("Backup created!", args, True)
    
def run_correction(obs, working_dir, args):
    ut.Print("Saving correction frames...", args, True)
    cor.save_correction(obs, working_dir, args)
    ut.Print("Correction frames saved!", args, True)

def run_reduce(obs, working_dir, args):
    ut.Print("Reducing images...", args, True)
    red.reduce_imgs(obs, working_dir, args)
    ut.Print("Images reduced!", args, True)
    
def run_pending(obs, working_dir, args):
    ut.Print("Rerunning pending reductions...", args, True)
    pd.rerun_pending(obs, working_dir, args)
    ut.Print("Pending list updated!", args, True)

def main():
    # Retrieve the arguments, and the specified targets
    args, targets = load_args()
    
    # Loop over every target that was found and perform the actions 
    # that were specified in the command line arguments
    for target in targets:
        ut.Print(f"Currently handling: {target}", args, True)
        
        # Construct a working dir where all *new/altered* data goes
        working_dir_name = os.path.relpath(target, cst.tele_path)
        working_dir = os.path.join(cst.base_path, working_dir_name)
        
        # Extra check if the working dir already exists, else create it
        if os.path.isdir(working_dir):
            # TODO: this means this data probably has been handled already...
            # BUT: not sure what exact operations have been carried out
            # Maybe we can put some sort of log in each handled dir, containing
            # a list of all actions that were already performed? Seems better 
            # than just 'checking' manually what has or hasn't been done yet.
            #print("Folder existed")
            pass
        else:
            os.makedirs(working_dir)
        
        # Create observation object for this target
        obs = ut.get_observation(target, args)
    
        # Perform core functions
        if args.backup: run_backup(obs, working_dir, args)
        if args.correction: run_correction(obs, working_dir, args)
        if args.reduce: run_reduce(obs, working_dir, args)
        if args.pending: run_pending(obs, working_dir, args)
            
        # Execute plugins
        if args.modules: run_plugins(obs, working_dir, args)
    
if __name__ == "__main__":
    # placeholder
    # data_dir = "/net/vega/data/users/observatory/images/200417/STL-6303E/i/"
    date = "20-04-20"
    #date = "20-05-05"
    #22 april
    
    sys.argv = ["controller.py", "-d", date, "-bcrv"]
#    sys.argv = ["controller.py", "-d", date, "-bv"]
#     sys.argv = ["controller.py", "-u", "-rv"]
    main()

16:28:50.195245 Found data in 1 subdir(s)
16:28:50.195353 Currently handling: /net/vega/data/users/observatory/images/200420/STL-6303E/i
16:28:50.199479 Creating Observation object...
16:28:51.316125 Found 123 fits files
16:28:51.316197 Found 27 bias frames
16:28:51.316234 Found 4 dark frames
16:28:51.316282 Found 15 flat fields
16:28:51.317370 Found 77 light frames
16:28:51.317416 Observation object initialized
16:28:51.317458 Creating backup...
16:28:51.585958 Copied file to backup: 200420_Li_.00000001.23h59m03.8s_00d06m04sS.BIAS.FIT
16:28:51.856866 Copied file to backup: 200420_Li_.00000002.23h59m03.8s_00d06m04sS.BIAS.FIT
16:28:52.128577 Copied file to backup: 200420_Li_.00000003.23h59m02.8s_00d06m04sS.BIAS.FIT
16:28:52.404434 Copied file to backup: 200420_Li_.00000004.23h59m02.8s_00d06m04sS.BIAS.FIT
16:28:52.674860 Copied file to backup: 200420_Li_.00000005.23h59m02.8s_00d06m04sS.BIAS.FIT
16:28:52.934967 Copied file to backup: 200420_Li_.00000006.23h59m02.8s_00d06m04sS.BIAS.FIT
16:

16:29:18.640974 Copied file to backup: 200420_Li_.00000100.Entered_Coordinates.FIT
16:29:18.913733 Copied file to backup: 200420_Li_.00000101.Entered_Coordinates.FIT
16:29:19.187115 Copied file to backup: 200420_Li_.00000102.Entered_Coordinates.FIT
16:29:19.461385 Copied file to backup: 200420_Li_.00000104.Entered_Coordinates.FIT
16:29:19.731090 Copied file to backup: 200420_Li_.00000105.Entered_Coordinates.FIT
16:29:20.001624 Copied file to backup: 200420_Li_.00000106.Entered_Coordinates.FIT
16:29:20.280226 Copied file to backup: 200420_Li_.00000107.M_3.FIT
16:29:20.549899 Copied file to backup: 200420_Li_.00000108.M_3.FIT
16:29:20.817101 Copied file to backup: 200420_Li_.00000109.M_3.FIT
16:29:21.088022 Copied file to backup: 200420_Li_.00000110.M_3.FIT
16:29:21.361227 Copied file to backup: 200420_Li_.00000111.M_3.FIT
16:29:21.634590 Copied file to backup: 200420_Li_.00000112.M_3.FIT
16:29:21.909768 Copied file to backup: 200420_Li_.00000113.M_3.FIT
16:29:22.180947 Copied file to ba

16:30:16.025229 /net/dataserver3/data/users/noelstorr/blaauwpipe/200420/STL-6303E/i/Reduced/200420_Li_.00000070.NGC_5053.FIT created
16:30:17.146176 /net/dataserver3/data/users/noelstorr/blaauwpipe/200420/STL-6303E/i/Reduced/200420_Li_.00000082.M_53.FIT created
16:30:17.902522 /net/dataserver3/data/users/noelstorr/blaauwpipe/200420/STL-6303E/i/Reduced/200420_Li_.00000083.M_53.FIT created
16:30:18.641773 /net/dataserver3/data/users/noelstorr/blaauwpipe/200420/STL-6303E/i/Reduced/200420_Li_.00000084.M_53.FIT created
16:30:19.399451 /net/dataserver3/data/users/noelstorr/blaauwpipe/200420/STL-6303E/i/Reduced/200420_Li_.00000085.M_53.FIT created
16:30:20.144426 /net/dataserver3/data/users/noelstorr/blaauwpipe/200420/STL-6303E/i/Reduced/200420_Li_.00000086.M_53.FIT created
16:30:20.875704 /net/dataserver3/data/users/noelstorr/blaauwpipe/200420/STL-6303E/i/Reduced/200420_Li_.00000087.M_53.FIT created
16:30:21.633911 /net/dataserver3/data/users/noelstorr/blaauwpipe/200420/STL-6303E/i/Reduced/2

In [9]:
import configparser

config = configparser.ConfigParser()
config['CORE'] = {}

enabled_plugins = {}
for plugin in loaded_plugins:
    enabled_plugins[type(plugin).__name__] = True
    
config['ENABLED PLUGINS'] = {type(plugin).__name__}

with open('config.ini', 'w') as configfile:
    config.write(configfile)

NameError: name 'loaded_plugins' is not defined